<a href="https://colab.research.google.com/github/whrc/interactive-sam2/blob/main/interactive_label_sam2_(v1_0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# notebooks/interactive_label_sam2.ipynb

# ==============================================================================
# Cell 1: One-Time Environment Setup
# (Run this cell only ONCE per Colab session)
# ==============================================================================

# --- 1.1 Install necessary libraries ---
# We install ipycanvas for robust interactive plotting in Colab.
# print("Installing necessary libraries...")
# !pip install rasterio ipycanvas --quiet
# print("Installation complete.")

# --- 1.2 Clone the GitHub Repository ---
import os
if not os.path.exists('interactive-sam2'):
    print("Cloning the project repository from GitHub...")
    !git clone https://github.com/whrc/interactive-sam2.git
    %cd interactive-sam2
    !git lfs pull
    %cd ..
    print("Repository cloned and LFS files downloaded.")
else:
    print("Repository already exists. Skipping clone.")

# ==============================================================================
# Cell 2: Main Application
# (You can restart the kernel before running this cell if you encounter issues)
# ==============================================================================

# --- 2.1 Import Libraries ---
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
import sys
import numpy as np
from PIL import Image
import pandas as pd
import datetime
import geopandas as gpd
import io
import rasterio
from ipycanvas import Canvas, hold_canvas

# --- 2.2 Authenticate and Mount Drives ---
try:
    from google.colab import auth, drive
    print("Authenticating with Google Cloud...")
    auth.authenticate_user()
    print("Authentication successful.")

    print("Mounting Google Drive...")
    drive.mount('/content/drive')
    print("Google Drive mounted successfully at /content/drive.")

except ImportError:
    print("Not running in Google Colab. Skipping authentication and drive mount.")

# --- 2.3 Add Source Code to Python Path ---
project_root = Path("./interactive-sam2")
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.interactive_label_sam2.gcs_utils import GCSImageLoader, load_correspondence_data, get_image_info_for_uid
from src.interactive_label_sam2.model import SAM2Model
from src.interactive_label_sam2.data_management import load_and_filter_arts_data

# --- 2.4 Application State and UI Definition ---
print("\n--- Defining Application State and UI Components ---")
DRIVE_ROOT = Path("/content/drive/MyDrive/Interactive_sam")
OUTPUT_DIR = DRIVE_ROOT / "test"
MANIFEST_PATH = OUTPUT_DIR / "rts_labeling_manifest.csv"

APP_STATE = {
    "worker_id": None, "manifest_df": None, "arts_gdf": None,
    "correspondence_gdf": None, "gcs_loader": None, "sam_model": None,
    "current_uid": None, "current_image_array": None, "current_display_image": None,
    "current_tile_profile": None, "current_mask": None, "prompts": [],
    "canvas": None
}

worker_id_input = widgets.Text(value='worker_01', description='Worker ID:', layout=widgets.Layout(width='300px'))
start_button = widgets.Button(description="Start Labeling Session", button_style='success')
next_uid_button = widgets.Button(description="Get Next Unprocessed RTS", icon='forward', layout=widgets.Layout(width='250px'))
uid_display = widgets.HTML(value="<h3>Current UID: None</h3>")
prompt_toggle = widgets.ToggleButtons(options=['Positive', 'Negative'], description='Prompt Type:', button_style='info')
run_sam_button = widgets.Button(description="Generate Mask", icon='magic', button_style='primary')
clear_prompts_button = widgets.Button(description="Clear Prompts", icon='trash')
save_button = widgets.Button(description="Save Final Mask", icon='save', button_style='success')
reject_bad_image_button = widgets.Button(description="Reject (Bad Image)", icon='times-circle', button_style='danger')
reject_no_feature_button = widgets.Button(description="Reject (No Feature)", icon='ban', button_style='warning')

controls_col1 = widgets.VBox([uid_display, prompt_toggle])
controls_col2 = widgets.VBox([run_sam_button, clear_prompts_button])
prompting_controls = widgets.HBox([controls_col1, controls_col2])
finalization_controls = widgets.HBox([save_button, reject_bad_image_button, reject_no_feature_button])
main_controls = widgets.VBox([next_uid_button, prompting_controls, finalization_controls])
login_ui = widgets.VBox([worker_id_input, start_button])
main_app_ui = widgets.VBox([main_controls])

print("UI components defined.")

# --- 2.5 Application Logic ---

def on_mouse_down(x, y):
    """Callback function for when the user clicks on the ipycanvas."""
    label = 1 if prompt_toggle.value == 'Positive' else 0
    APP_STATE["prompts"].append({'coords': (int(x), int(y)), 'label': label})
    redraw_canvas(image_to_show=APP_STATE["current_display_image"], mask=APP_STATE["current_mask"])
    print(f"Added {prompt_toggle.value} prompt at pixel ({int(x)}, {int(y)})")

def redraw_canvas(image_to_show=None, mask=None):
    """Clears and redraws the ipycanvas with the image, prompts, and mask."""
    canvas = APP_STATE["canvas"]
    with hold_canvas(canvas):
        canvas.clear()
        if image_to_show is not None:
            # The put_image_data method is the correct way to draw a NumPy array
            canvas.put_image_data(image_to_show, 0, 0)

            if mask is not None:
                # Create a transparent overlay for the mask
                mask_rgba = np.zeros((mask.shape[0], mask.shape[1], 4), dtype=np.uint8)
                mask_rgba[mask == 1] = [255, 0, 0, 100] # Red with alpha
                canvas.put_image_data(mask_rgba, 0, 0)

            # Draw prompts on top
            for prompt in APP_STATE["prompts"]:
                x, y = prompt['coords']
                canvas.fill_style = 'lime' if prompt['label'] == 1 else 'red'
                canvas.fill_rect(x - 3, y - 3, 7, 7)

def on_run_sam_button_clicked(b):
    if not APP_STATE["prompts"]:
        print("Please add at least one prompt point.")
        return
    print("Generating mask...")
    points = [p['coords'] for p in APP_STATE["prompts"]]
    labels = [p['label'] for p in APP_STATE["prompts"]]
    # The model expects an (H, W, C) image, which current_display_image already is (as RGBA)
    mask_array = APP_STATE["sam_model"].run_inference(APP_STATE["current_display_image"][:,:,:3], points, labels)
    APP_STATE["current_mask"] = mask_array
    redraw_canvas(image_to_show=APP_STATE["current_display_image"], mask=mask_array)
    print("Mask generated and added to plot.")

def on_clear_prompts_button_clicked(b):
    APP_STATE["prompts"] = []
    APP_STATE["current_mask"] = None
    if APP_STATE.get("current_display_image") is not None:
        redraw_canvas(image_to_show=APP_STATE["current_display_image"])
    print("All prompts and masks have been cleared.")

def finalize_labeling(status: str):
    uid = APP_STATE.get("current_uid")
    if not uid: return
    print(f"Finalizing UID {uid} with status: {status}")

    if status == 'completed':
        if APP_STATE.get("current_mask") is None:
            print("Error: Cannot save. Please generate a mask first.", file=sys.stderr)
            return
        r, g, b = APP_STATE["current_image_array"][:3]
        mask = APP_STATE["current_mask"]
        output_array = np.stack([r, g, b, mask])
        profile = APP_STATE["current_tile_profile"]
        profile.update(count=4, dtype=rasterio.uint8)
        output_filename = f"{uid}_mask.tif"
        output_path = OUTPUT_DIR / output_filename
        print(f"Saving 4-channel GeoTIFF to: {output_path}")
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(output_array.astype(rasterio.uint8))
        APP_STATE["manifest_df"].loc[APP_STATE["manifest_df"]['uid'] == uid, 'output_filename'] = output_filename
    else:
        APP_STATE["manifest_df"].loc[APP_STATE["manifest_df"]['uid'] == uid, 'output_filename'] = ''

    APP_STATE["manifest_df"].loc[APP_STATE["manifest_df"]['uid'] == uid, 'labeling_status'] = status
    APP_STATE["manifest_df"].loc[APP_STATE["manifest_df"]['uid'] == uid, 'end_time_utc'] = datetime.datetime.utcnow().isoformat()
    APP_STATE["manifest_df"].to_csv(MANIFEST_PATH, index=False)
    print("Manifest updated in Google Drive. Loading next UID...")
    load_next_uid()

def load_next_uid():
    on_clear_prompts_button_clicked(None)
    manifest_df = APP_STATE["manifest_df"]
    worker_id = APP_STATE["worker_id"]
    worker_in_progress = manifest_df[(manifest_df['labeling_status'] == 'in_progress') & (manifest_df['worker_id'] == worker_id)]

    next_uid = None
    if not worker_in_progress.empty:
        next_uid = worker_in_progress.iloc[0]['uid']
    else:
        unprocessed_series = manifest_df[manifest_df['labeling_status'] == 'unprocessed']
        if not unprocessed_series.empty:
            next_uid = unprocessed_series.iloc[0]['uid']
        else:
            uid_display.value = "<h3>All UIDs Processed!</h3>"; redraw_canvas(); return

    feature_polygons = APP_STATE["arts_gdf"][APP_STATE["arts_gdf"]['UID'] == next_uid]
    if feature_polygons.empty:
        finalize_labeling('rejected_not_in_arts'); return

    true_centroid = feature_polygons.geometry.unary_union.centroid
    image_info_list = get_image_info_for_uid(next_uid, APP_STATE["correspondence_gdf"])
    if not image_info_list: return

    image_info = image_info_list[0]
    centroid_gs = gpd.GeoSeries([true_centroid], crs=APP_STATE["arts_gdf"].crs)
    buffer_distance = 384
    aoi_gdf = gpd.GeoDataFrame(geometry=centroid_gs.buffer(buffer_distance))

    gcs_paths = APP_STATE["gcs_loader"].find_image_paths([image_info])
    tile_data = APP_STATE["gcs_loader"].get_tile_from_paths(gcs_paths, aoi_gdf)

    if not tile_data:
        finalize_labeling('rejected_bad_imagery'); return

    APP_STATE["current_uid"] = next_uid
    uid_display.value = f"<h3>Current UID: {next_uid}</h3>"
    manifest_df.loc[manifest_df['uid'] == next_uid, 'labeling_status'] = 'in_progress'
    manifest_df.loc[manifest_df['uid'] == next_uid, 'worker_id'] = worker_id
    manifest_df.loc[manifest_df['uid'] == next_uid, 'start_time_utc'] = datetime.datetime.utcnow().isoformat()
    manifest_df.to_csv(MANIFEST_PATH, index=False)

    image_array, profile = tile_data
    APP_STATE["current_image_array"] = image_array
    APP_STATE["current_tile_profile"] = profile

    rgb_array = image_array[:3]
    if np.max(rgb_array) > 0:
        p2, p98 = np.percentile(rgb_array, (2, 98))
        rgb_stretched = np.clip((rgb_array - p2) * 255.0 / (p98 - p2), 0, 255).astype(np.uint8)
    else:
        rgb_stretched = np.zeros((image_array.shape[1], image_array.shape[2], 3), dtype=np.uint8)

    # Transpose the stretched RGB data to (H, W, C) format
    rgb_display = np.transpose(rgb_stretched, (1, 2, 0))

    # Create the RGBA image for the canvas
    rgba_display = np.zeros((rgb_display.shape[0], rgb_display.shape[1], 4), dtype=np.uint8)
    # Correctly assign the (H, W, C) data to the RGBA container
    rgba_display[:, :, :3] = rgb_display
    rgba_display[:, :, 3] = 255 # Set alpha to fully opaque

    APP_STATE["current_display_image"] = rgba_display

    APP_STATE["canvas"].width = rgba_display.shape[1]
    APP_STATE["canvas"].height = rgba_display.shape[0]
    redraw_canvas(image_to_show=rgba_display)

def on_start_button_clicked(b):
    clear_output()

    canvas = Canvas(width=800, height=600)
    canvas.on_mouse_down(on_mouse_down)
    APP_STATE["canvas"] = canvas

    display(main_app_ui)
    display(canvas)

    print("--- Initializing Application... ---")
    APP_STATE["worker_id"] = worker_id_input.value

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Output directory set to: {OUTPUT_DIR}")

    if not MANIFEST_PATH.exists():
        print(f"Manifest not found in Drive. Copying from repository...")
        repo_manifest = project_root / "manifest.csv"
        if repo_manifest.exists():
            import shutil
            shutil.copy(repo_manifest, MANIFEST_PATH)
            print("Manifest copied successfully.")
        else:
            print("FATAL: Could not find manifest.csv in the repository.", file=sys.stderr)
            return

    print(f"Loading manifest from: {MANIFEST_PATH}")
    APP_STATE["manifest_df"] = pd.read_csv(MANIFEST_PATH)
    print("Manifest loaded.")

    correspondence_path = project_root / "data" / "raw" / "planet_basemaps_rts_polygon_basemap_correspondence.geojson"
    APP_STATE["correspondence_gdf"] = load_correspondence_data(correspondence_path)

    arts_path = project_root / "data" / "raw" / "ARTS_main_dataset_v.3.1.0.geojson"
    APP_STATE["arts_gdf"] = load_and_filter_arts_data(arts_path)

    APP_STATE["gcs_loader"] = GCSImageLoader(project_id="abruptthawmapping", bucket_name="abrupt_thaw", search_prefix="planet_basemaps/global_quarterly_COGs")
    APP_STATE["sam_model"] = SAM2Model(model_name="facebook/sam-vit-base")

    print("\nInitialization complete.")
    print(f"Welcome, {APP_STATE['worker_id']}!")
    print("Click 'Get Next Unprocessed RTS' to begin.")

# --- 2.6 Wire up the buttons to their functions ---
start_button.on_click(on_start_button_clicked)
next_uid_button.on_click(lambda b: load_next_uid())
run_sam_button.on_click(on_run_sam_button_clicked)
clear_prompts_button.on_click(on_clear_prompts_button_clicked)
save_button.on_click(lambda b: finalize_labeling('completed'))
reject_bad_image_button.on_click(lambda b: finalize_labeling('rejected_bad_imagery'))
reject_no_feature_button.on_click(lambda b: finalize_labeling('rejected_no_feature'))

# ==============================================================================
# Phase 4: Display the Initial UI
# ==============================================================================
print("\n--- Displaying User Interface ---")
display(login_ui)



Canvas(height=600, width=800)

--- Initializing Application... ---
Output directory set to: /content/drive/MyDrive/Interactive_sam/test
Loading manifest from: /content/drive/MyDrive/Interactive_sam/test/rts_labeling_manifest.csv
Manifest loaded.
Correspondence file loaded successfully with CRS: EPSG:3413
--- Initializing GCSImageLoader ---
Pre-fetching all blob paths from gs://abrupt_thaw/planet_basemaps/global_quarterly_COGs...
Found 42803 total .tif files. Ready for fast searching.
--- GCSImageLoader Initialized Successfully ---

--- Initializing SAM2 Model ---
Using device: cuda
Loading model: facebook/sam-vit-base...
--- SAM2 Model Initialized Successfully ---


Initialization complete.
Welcome, worker_01!
Click 'Get Next Unprocessed RTS' to begin.
All prompts and masks have been cleared.
Found 1 matching image path(s): ['abrupt_thaw/planet_basemaps/global_quarterly_COGs/global_quarterly_2020q3_mosaic/global_quarterly_2020q3_mosaic_337-1619_338-1619/global_quarterly_2020q3_mosaic_337-1622_quad.tif']


/tmp/ipython-input-7-1689695230.py:202: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  true_centroid = feature_polygons.geometry.unary_union.centroid



--- Processing Image: global_quarterly_2020q3_mosaic_337-1622_quad.tif ---
Vector CRS: EPSG:3413 | Raster CRS: EPSG:3857
CRS mismatch detected. Reprojecting AOI polygon...
Reading window with bounds: [-13434602.62026185  11715586.57996958 -13432126.95998975
  11718063.85091601]
Successfully read tile from abrupt_thaw/planet_basemaps/global_quarterly_COGs/global_quarterly_2020q3_mosaic/global_quarterly_2020q3_mosaic_337-1619_338-1619/global_quarterly_2020q3_mosaic_337-1622_quad.tif
Added Positive prompt at pixel (140, 125)
Added Positive prompt at pixel (248, 208)
Added Negative prompt at pixel (258, 149)
Added Negative prompt at pixel (208, 109)
Generating mask...
Mask generated and added to plot.
Added Negative prompt at pixel (262, 122)
Added Negative prompt at pixel (224, 70)
Added Negative prompt at pixel (84, 78)
Added Negative prompt at pixel (82, 156)
Generating mask...
Mask generated and added to plot.
All prompts and masks have been cleared.
Added Positive prompt at pixel (17

/tmp/ipython-input-7-1689695230.py:202: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  true_centroid = feature_polygons.geometry.unary_union.centroid


...


--- Processing Image: global_quarterly_2020q3_mosaic_337-1622_quad.tif ---
Vector CRS: EPSG:3413 | Raster CRS: EPSG:3857
CRS mismatch detected. Reprojecting AOI polygon...
Reading window with bounds: [-13429893.04173239  11713991.8056567  -13427417.91941439
  11716468.5390647 ]
Successfully read tile from abrupt_thaw/planet_basemaps/global_quarterly_COGs/global_quarterly_2020q3_mosaic/global_quarterly_2020q3_mosaic_337-1619_338-1619/global_quarterly_2020q3_mosaic_337-1622_quad.tif
Finalizing UID bec00682-28d7-5761-8731-d055380b2831 with status: rejected_no_feature
Manifest updated in Google Drive. Loading next UID...
All prompts and masks have been cleared.
Found 1 matching image path(s): ['abrupt_thaw/planet_basemaps/global_quarterly_COGs/global_quarterly_2020q3_mosaic/global_quarterly_2020q3_mosaic_338-1620_343-1639/global_quarterly_2020q3_mosaic_338-1622_quad.tif']


/tmp/ipython-input-7-1689695230.py:202: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  true_centroid = feature_polygons.geometry.unary_union.centroid



--- Processing Image: global_quarterly_2020q3_mosaic_338-1622_quad.tif ---
Vector CRS: EPSG:3413 | Raster CRS: EPSG:3857
CRS mismatch detected. Reprojecting AOI polygon...
Reading window with bounds: [-13424153.77919727  11710911.45709212 -13421679.75495328
  11713387.09319063]
Successfully read tile from abrupt_thaw/planet_basemaps/global_quarterly_COGs/global_quarterly_2020q3_mosaic/global_quarterly_2020q3_mosaic_338-1620_343-1639/global_quarterly_2020q3_mosaic_338-1622_quad.tif
Added Positive prompt at pixel (201, 144)
Added Positive prompt at pixel (201, 177)
Added Positive prompt at pixel (181, 222)
Added Positive prompt at pixel (156, 303)
Added Positive prompt at pixel (132, 380)
Added Positive prompt at pixel (69, 343)
Added Positive prompt at pixel (111, 280)
Added Positive prompt at pixel (66, 250)
Added Positive prompt at pixel (124, 222)
Added Positive prompt at pixel (152, 150)
Added Negative prompt at pixel (294, 98)
Added Negative prompt at pixel (292, 189)
Added Negati

/tmp/ipython-input-7-1689695230.py:202: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  true_centroid = feature_polygons.geometry.unary_union.centroid



--- Processing Image: global_quarterly_2020q3_mosaic_338-1622_quad.tif ---
Vector CRS: EPSG:3413 | Raster CRS: EPSG:3857
CRS mismatch detected. Reprojecting AOI polygon...
Reading window with bounds: [-13422322.62525019  11713507.42040172 -13419847.56393087
  11715984.09300396]
Successfully read tile from abrupt_thaw/planet_basemaps/global_quarterly_COGs/global_quarterly_2020q3_mosaic/global_quarterly_2020q3_mosaic_338-1620_343-1639/global_quarterly_2020q3_mosaic_338-1622_quad.tif
